# README  
This script is writed to validate the useful of Att-BLSTM on XX file  
THe best  percision result of the network is 0.575
Not very good

In [1]:
import numpy as np
import pandas as pd
import sys
import os

os.environ['KERAS_BACKEND']='theano'
from keras.engine.topology import Layer
from keras import initializations
from keras import backend as K

from collections  import defaultdict
import pandas as pd
import numpy as np

Using Theano backend.
WARNING (theano.gof.compilelock): Overriding existing lock by dead process '16472' (I am process '16796')
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5105)


In [2]:
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):


        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        '''
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])


#Attention GRU network
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        '''   
        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [3]:
datafile ='/home/yinqijin/WorkSpace/2.RNA_Structure_Profile/Orig_data/TRAIN_FILE.TXT'
f = open(datafile,'r')
sentenses = []
labels = []

i =0
for line in f:
    if i%4==0:
        sen = line.split('"')[1]
        #print sen
        sen = sen.replace('<e1>','')
        sen = sen.replace('</e1>','')
        sen = sen.replace('<e2>','')
        sen = sen.replace('</e2>','')
        #print sen
        sentenses.append(sen)
    elif i%4==1:
        labels.append(line.splitlines()[0])
    elif i%4==2:
        #This Commment , it's useless,pass
        pass
    else:
        pass
    
    i += 1
    ''' 
    if i>=12:
        print sentenses
        print labels
        break
   '''
    
f.close()
labels = np.array(labels)
sentenses = np.array(sentenses)

print len(sentenses),'==',len(labels)

8000 == 8000


In [4]:
print labels[:3]
print sentenses[:3]


['Component-Whole(e2,e1)' 'Other' 'Instrument-Agency(e2,e1)']
[ 'The system as described above has its greatest application in an arrayed configuration of antenna elements.'
 'The child was carefully wrapped and bound into the cradle by means of a cord.'
 'The author of a keygen uses a disassembler to look at the raw assembly code.']


In [5]:
label_name = np.unique(labels)
print  'labes kind(include \'Other\'): ',len(label_name) 
#print np.array(label_name)

label_dict = defaultdict()
for item in label_name:
    label_dict[item] = len(label_dict)

    
func = lambda item:label_dict[item]
labels_ = pd.Series(labels)
labels_ =labels_.apply(func)
#labes = labels_.tolist()
labels = np.array(labels_)
print labels

labes kind(include 'Other'):  19
[ 3 16 11 ...,  6 16 18]


In [6]:
np.random.seed(1234)
index = range(len(labels))
np.random.shuffle(index)
#print index
#print labels[:5],sentenses[:5]
labels = labels[index]
sentenses = sentenses[index]
#print labels[:5],sentenses[:5]

In [7]:
LongestSen = max(sentenses,key= lambda x :len(x))
print len(LongestSen)

492


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

In [9]:
MAX_SEQUENCE_LENGTH = 500
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [10]:
GLOVE_DIR = '/home/yinqijin/WorkSpace/2.RNA_Structure_Profile/Orig_data/glove.6B.100d.txt'
embeddings_index = {}
f = open(GLOVE_DIR,'r')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [11]:
# Tokenizer.fit_on_sequences Tokenizer.fit_on_texts,  any different??
tokenizer = Tokenizer(nb_words= MAX_NB_WORDS)
tokenizer.fit_on_texts(sentenses)
sequences = tokenizer.texts_to_sequences(sentenses)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 19391 unique tokens.


In [12]:
word_index = tokenizer.word_index
print word_index.keys()[:4]
print len(word_index.keys())

['writings', 'yellow', 'narcotic', 'four']
19391


In [13]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)
model_labels = to_categorical(np.asarray(labels))   #one-hot 编码labels

In [14]:
print data[1],data.shape[0]
print model_labels[1]

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0  

In [15]:
nb_validation_samples = int(VALIDATION_SPLIT*data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = model_labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = model_labels[-nb_validation_samples:]

print('Traing and validation set number of positive and negative reviews')
print y_train.shape[0]
print y_val.shape[0]

Traing and validation set number of positive and negative reviews
6400
1600


In [16]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# Make Network

In [17]:
from keras.layers import Embedding
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout

In [18]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True,
                           dropout =0.3)



sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm = Bidirectional(LSTM(100, return_sequences=True))(embedded_sequences)
l_lstm_drop = Dropout(0.3)(l_lstm)
l_att = AttLayer()(l_lstm_drop)
l_att_drop = Dropout(0.5)(l_att)
preds = Dense(len( model_labels[0]), activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_att)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='Adadelta',
              metrics=['categorical_accuracy'])

print("model fitting - attention LSTM network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=40, batch_size=10)

model fitting - attention LSTM network
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 500)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 500, 100)      1939200     input_1[0][0]                    
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 500, 200)      160800      embedding_1[0][0]                
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 500, 200)      0           bidirectional_1[0][0]            
____________________________________________________

In [19]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True,
                           dropout=0.3)

In [20]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm = Bidirectional(LSTM(100, return_sequences=True),merge_mode='sum')(embedded_sequences)
l_att = AttLayer()(l_lstm)
preds = Dense(len( model_labels[0]), activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_att)


In [22]:
model = Model(sequence_input, preds)
model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['acc','recall','fbeta_score'])
print("model fitting - attention LSTM network")
#model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),nb_epoch=40, batch_size=10)

model fitting - attention LSTM network

['nvcc', '-shared', '-O3', '--maxrregcount=32', '-arch=sm_52', '-m64', '-Xcompiler', '-fno-math-errno,-Wno-unused-label,-Wno-unused-variable,-Wno-write-strings,-DCUDA_NDARRAY_CUH=c72d035fdf91890f3b36710688069b2e,-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION,-fPIC,-fvisibility=hidden', '-Xlinker', '-rpath,/home/yinqijin/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.2.1511-Core-x86_64-2.7.12-64/cuda_ndarray', '-I/home/yinqijin/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.2.1511-Core-x86_64-2.7.12-64/cuda_ndarray', '-I/prog/nvidia/cuda/default/include', '-I/home/yinqijin/.local/lib/python2.7/site-packages/numpy/core/include', '-I/home/yinqijin/anaconda2/include/python2.7', '-I/home/yinqijin/Theano/theano/gof', '-I/home/yinqijin/Theano/theano/sandbox/cuda', '-L/home/yinqijin/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.2.1511-Core-x86_64-2.7.12-64/cuda_ndarray', '-L/home/yinqijin/anaconda2/lib', '-o', '

1 #include <Python.h>
2 #include <iostream>
3 #include "theano_mod_helper.h"
4 #include "cuda_ndarray.cuh"
5 //////////////////////
6 ////  Support Code
7 //////////////////////
8 
9 
10     namespace {
11     struct __struct_compiled_op_3a084fd032db206a67ca8121a24aa32e {
12         PyObject* __ERROR;
13 
14         PyObject* storage_V3;
15 PyObject* storage_V1;
16         
17 
18         __struct_compiled_op_3a084fd032db206a67ca8121a24aa32e() {
19             // This is only somewhat safe because we:
20             //  1) Are not a virtual class
21             //  2) Do not use any virtual classes in the members
22             //  3) Deal with mostly POD and pointers
23 
24             // If this changes, we would have to revise this, but for
25             // now I am tired of chasing segfaults because
26             // initialization code had an error and some pointer has
27             // a junk value.
28             memset(this, 0, sizeof(*this));
29         }
30         ~__struct

Exception: ('The following error happened while compiling the node', GpuElemwise{RoundHalfToEven}[(0, 0)](GpuElemwise{clip,no_inplace}.0), '\n', 'nvcc return status', 2, 'for cmd', 'nvcc -shared -O3 --maxrregcount=32 -arch=sm_52 -m64 -Xcompiler -fno-math-errno,-Wno-unused-label,-Wno-unused-variable,-Wno-write-strings,-DCUDA_NDARRAY_CUH=c72d035fdf91890f3b36710688069b2e,-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION,-fPIC,-fvisibility=hidden -Xlinker -rpath,/home/yinqijin/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.2.1511-Core-x86_64-2.7.12-64/cuda_ndarray -I/home/yinqijin/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.2.1511-Core-x86_64-2.7.12-64/cuda_ndarray -I/prog/nvidia/cuda/default/include -I/home/yinqijin/.local/lib/python2.7/site-packages/numpy/core/include -I/home/yinqijin/anaconda2/include/python2.7 -I/home/yinqijin/Theano/theano/gof -I/home/yinqijin/Theano/theano/sandbox/cuda -L/home/yinqijin/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-ce